In [2]:
# Data Collection
import nltk
nltk.download('gutenberg')

from nltk.corpus import gutenberg
import pandas as pd

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


In [3]:
# Load dataset
data = gutenberg.raw('shakespeare-hamlet.txt')

with open('hamlet.txt', 'w') as file:
    file.write(data)

In [4]:
# Data Preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [5]:
with open('hamlet.txt', 'r') as file:
    text = file.read().lower()

In [6]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
total_words

4818

In [7]:
# Create input sequences

input_sequences = []
for line in text.split('\n'):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

In [8]:
input_sequences

[[1, 687],
 [1, 687, 4],
 [1, 687, 4, 45],
 [1, 687, 4, 45, 41],
 [1, 687, 4, 45, 41, 1886],
 [1, 687, 4, 45, 41, 1886, 1887],
 [1, 687, 4, 45, 41, 1886, 1887, 1888],
 [1180, 1889],
 [1180, 1889, 1890],
 [1180, 1889, 1890, 1891],
 [57, 407],
 [57, 407, 2],
 [57, 407, 2, 1181],
 [57, 407, 2, 1181, 177],
 [57, 407, 2, 1181, 177, 1892],
 [407, 1182],
 [407, 1182, 63],
 [408, 162],
 [408, 162, 377],
 [408, 162, 377, 21],
 [408, 162, 377, 21, 247],
 [408, 162, 377, 21, 247, 882],
 [18, 66],
 [451, 224],
 [451, 224, 248],
 [451, 224, 248, 1],
 [451, 224, 248, 1, 30],
 [408, 407],
 [451, 25],
 [408, 6],
 [408, 6, 43],
 [408, 6, 43, 62],
 [408, 6, 43, 62, 1893],
 [408, 6, 43, 62, 1893, 96],
 [408, 6, 43, 62, 1893, 96, 18],
 [408, 6, 43, 62, 1893, 96, 18, 566],
 [451, 71],
 [451, 71, 51],
 [451, 71, 51, 1894],
 [451, 71, 51, 1894, 567],
 [451, 71, 51, 1894, 567, 378],
 [451, 71, 51, 1894, 567, 378, 80],
 [451, 71, 51, 1894, 567, 378, 80, 3],
 [451, 71, 51, 1894, 567, 378, 80, 3, 273],
 [451, 71

In [9]:
# Pad Sequences
max_sequence_len = max([len(x) for x in input_sequences])
max_sequence_len

14

In [10]:
input_sequences = np.array(pad_sequences(input_sequences, maxlen = max_sequence_len))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [11]:
# create prepdictors and label
X, y = input_sequences[:, :-1], input_sequences[:, -1]

In [12]:
X

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       ...,
       [   0,    0,    0, ...,  687,    4,   45],
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4]], dtype=int32)

In [13]:
y

array([ 687,    4,   45, ..., 1047,    4,  193], dtype=int32)

In [14]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y, num_classes = total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101)

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, GRU

In [17]:
# Train LSTM RNN

model_lstm = Sequential(
    [Embedding(total_words, 100, input_length = max_sequence_len-1),
     LSTM(150, return_sequences=True),
     Dropout(0.2),
     LSTM(100),
     Dense(total_words, activation='softmax')]
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [18]:
model_lstm.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
model_lstm.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
model_lstm.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 100)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 14s 11ms/step - accuracy: 0.0296 - loss: 7.1242 - val_accuracy: 0.0324 - val_loss: 6.7459
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0377 - loss: 6.4355 - val_accuracy: 0.0406 - val_loss: 6.8125
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.0450 - loss: 6.3003 - val_accuracy: 0.0459 - val_loss: 6.8395
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.0530 - loss: 6.1427 - val_accuracy: 0.0464 - val_loss: 6.8678
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.0558 - loss: 6.0372 - val_accuracy: 0.0544 - val_loss: 6.8995
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.0674 - loss: 5.8605 - val_accuracy: 0.0622 - val_loss: 6.9272
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 8s 10ms/step - accuracy: 0.0798 - loss: 5.7065 - val_accuracy: 0.0628 - val_loss: 7.0067
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.0861 - loss: 5.5305 

In [21]:
def predict_next_word(model, tokenizer, text, max_sequence_len):
    token_list = tokenizer.texts_to_sequences([text])[0]
    if len(token_list) >= max_sequence_len:
        token_list = token_list[:max_sequence_len-1]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose = 0)
    predicted_word_index = np.argmax(predicted)
    for word, index in tokenizer.word_index.items():
        if index == predicted_word_index:
            return word
    return None

In [32]:
input_text = 'To be or not to '
max_sequence_len = model_lstm.input_shape[1] + 1
predicted_word = predict_next_word(model_lstm, tokenizer, input_text, max_sequence_len)
print(f"Input Text: {input_text}")
print(f"Predicted Next Word: {predicted_word}")

Input Text: To be or not to 
Predicted Next Word: be


In [35]:
import pickle
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [24]:
# Train LSTM GRU

model_gru = Sequential()
model_gru.add(Embedding(total_words, 100, input_length = max_sequence_len-1))
model_gru.add(GRU(150, return_sequences=True))
model_gru.add(Dropout(0.2))
model_gru.add(GRU(100))
model_gru.add(Dense(total_words, activation='softmax'))

model_gru.compile(optimizer='adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
model_gru.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [25]:
model_gru.fit(X_train, y_train, validation_data=(X_test, y_test), epochs = 100)

Epoch 1/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.0319 - loss: 7.1338 - val_accuracy: 0.0324 - val_loss: 6.7452
Epoch 2/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - accuracy: 0.0380 - loss: 6.4724 - val_accuracy: 0.0406 - val_loss: 6.8262
Epoch 3/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0439 - loss: 6.3086 - val_accuracy: 0.0427 - val_loss: 6.8605
Epoch 4/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - accuracy: 0.0532 - loss: 6.1405 - val_accuracy: 0.0517 - val_loss: 6.8625
Epoch 5/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.0542 - loss: 6.0420 - val_accuracy: 0.0480 - val_loss: 6.9246
Epoch 6/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.0579 - loss: 5.8983 - val_accuracy: 0.0596 - val_loss: 6.9776
Epoch 7/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 10ms/step - accuracy: 0.0681 - loss: 5.7628 - val_accuracy: 0.0643 - val_loss: 7.0881
Epoch 8/100
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 8ms/step - accuracy: 0.0771 - loss: 5.6243 - va

In [30]:
input_text = 'To be or not to '
max_sequence_len = model_gru.input_shape[1] + 1
predicted_word = predict_next_word(model_gru, tokenizer, input_text, max_sequence_len)
print(f"Input Text: {input_text}")
print(f"Predicted Next Word: {predicted_word}")

Input Text: To be or not to 
Predicted Next Word: heare


In [34]:
model_lstm.save('model_lstm.keras')
model_gru.save('model_gru.keras')